In [11]:
# -*- coding: utf-8 -*-

import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

########################################
########### IMPORT DATA ################
########################################

#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

#E.head()

########################################
########### MODEL ######################
########################################

# Create an empty model
m1= gp.Model(name="NFL1")

N = E.shape[1]

# ADD DECISION VARIABLES HERE
x_ijk = m1.addVars(N,N,12,vtype = GRB.BINARY,name = 'x_ijk')

########################################
#### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD CONSTRAINTS HERE

#0. A team cannot play with itself
m1.addConstrs((x_ijk[i,i,k] == 0 for i in range(N) for k in range(12)),name = 'not_possible')

#1. Each team plays once a week and same teams can't play with each other
for k in range(12):
    for j in range(N):
        week_sum = 0
        for i in range(N):
            week_sum += x_ijk[i,j,k]+x_ijk[j,i,k]
        m1.addConstr(week_sum==1)
    
#2. All 12 games for each team is against a different team
m1.addConstrs((sum(x_ijk[i,j,k] + x_ijk[j,i,k] for k in range(12)) <= 1 for i in range(N) 
             for j in range(N)),name="different_opponent")
# no need of j because we are summing the values of i and j in weekly data frame. 

#3. Have max 6 home games and rest away
m1.addConstrs((sum(x_ijk[i,j,k] for j in range(N) for k in range(12)) <=6 for i in range(N)), name= "max_home_game")

#4. No 2 home games and no 2 away games
#6. no games in week 9
#- we cannot have a seperate constraint for no games in week 9 because we have a constraint 
#- for having one game each week, to adjust thae we will break the 2 home and away game condition to two parts
#- one for week 1st to 8th and second for week 10th to 12th but to counter this in programming we will skip including weeek 9 in the code 
#- here week 9-12 correspond to week 10-13 and 1-8 to week 1-8
#- loop is till 11 so it stays in bounds when done k+1
# we are skipping 9th week
#7/8 as home for A week 9 cannot be home according to constrs
#7/8 as away for A week 9 cannot be away according to constrs
#home home  |added later/ignore the existance|    home/away home/away away/home 
#7.   8.           9.          10.       11.  

#home games
for i in range(N):
    for k in range(6):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home<=2)

        
for i in range(N):
    for k in range(6):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for j in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home>=1)
        
#away
for j in range(N):
    for k in range(6):
        home = 0
        for i in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for i in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home<=2)

        
for j in range(N):
    for k in range(6):
        home = 0
        for i in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
    for k in range(8,10):
        home = 0
        for i in range(N):
            home += x_ijk[i,j,k]+x_ijk[i,j,k+1]+x_ijk[i,j,k+2]
        m1.addConstr(home>=1)
        
        

#5. Each teams plays 12 games
for j in range(N):
    total_games = 0
    for i in range(N):
        for k in range(12):
            total_games += x_ijk[i,j,k]+x_ijk[j,i,k]
    m1.addConstr(total_games==12)


# ADD OBJECTIVE FUNCTION HERE
objective = 0
for j in range(N):
    for i in range(N):
        for k in range(12):
            objective += x_ijk[i,j,k] * E.iloc[i,j]*2
            
m1.setObjective(objective,GRB.MINIMIZE)
#optimize the model
m1.optimize()

########################################
########### PRINT RESULTS ##############
########################################

# ADD PRINTING HERE:

# print the total distance travelled by all teams
print('\nValue of objective function: %g' % m1.objVal)

# print the optimal schedule for Cleveland Browns
#14:"Cleveland Browns"
rows=[]
for v in m1.getVars():
    if v.varName.find("14")!= -1 and v.x==1:
        rows.append([v.varName, v.x])
print('\n Schedule for Cleveland Browns')
df = pd.DataFrame(rows, columns=["v.varName", "v.x"])
print(df)


# Q3: print distance that each team travels

# Q4: print distance and schedule for Chicago Bears and Jacksonville Jaguars

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1392 rows, 6912 columns and 61632 nonzeros
Model fingerprint: 0x1df2a8f4
Variable types: 0 continuous, 6912 integer (6912 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 612 rows and 288 columns
Presolve time: 0.06s
Presolved: 780 rows, 6624 columns, 39744 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 4067 iterations, 0.38 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  151          - 165908.000      -     -    1s
H    0     0                    195134.00000 165908.000  15.0%     -    2s
H    0     0                    165908.00000 165908.000  0.00%     -    2s
     0     0 1

In [ ]:
####NOTE####

#Week 9 is to be considered in the questions after the solution.  